# 1. MySQL에서 table 가져오기

In [1]:
#!pip install mysql-connector-python

^C


In [5]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=d04859a0d82e41908d282308beee8da16060a3adedd8ebf9f97d0a657e2e3d28
  Stored in directory: c:\users\sey\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [11]:
# import mysql.connector
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np
import sys
import mysql.connector

In [12]:
cnx = mysql.connector.connect(user = 'root', password = 'mysql',
                             host = '127.0.0.1',
                             database = 'loveforpet')

In [13]:
cursor = cnx.cursor()


In [14]:
query = ("SELECT forpet_hash,address_name,place_name  FROM loveforpet.forpet_shop; ")

In [15]:
cursor.execute(query)

In [16]:
ordinary_table = pd.DataFrame(columns =['forpet_hash', 'address_name', 'place_name'])

In [17]:
ordinary_table.head()

,forpet_hash,address_name,place_name


In [18]:
for i, cor in enumerate(cursor):
    ordinary_table.loc[i] = cor

In [19]:
cursor.close()
cnx.close()

In [20]:
len(ordinary_table)

8582

In [21]:
ordinary_table.to_csv('mysql_forpet_shop.csv')

# 2. table 정보 이용해서 크롤링

### 2.1. crawrler 

In [20]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")

time.sleep(2)


element = driver.find_element_by_name('q')

element.clear()
element.send_keys('google map')
element.submit()

element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

In [21]:
# 여러개의  shop 목록에서 주소로 찾기
def find_shop_by_address(select_shop_html, shop_address):
    soup = BeautifulSoup(select_shop_html, 'html.parser')
    all_address =soup.find_all('span', attrs = {'class': "section-result-location"} )
    shop_index = 0
    shop_select_dict = {}
    for i, address in enumerate(all_address):
        print('address list', i, address.text)
        gu = shop_address[0] in address.text
        dong = shop_address[1] in address.text
        bungi = shop_address[2] in address.text
        if (gu and dong and bungi) or (dong and bungi):
            shop_select_dict[str(i+1)] = 'match'
        elif gu or dong:
            shop_select_dict[str(i+1)] = 'not match'
        else:
            pass
#     print(shop_select_dict)
    return shop_select_dict


In [22]:
# 주소와 total 별점 가져오기
def get_totalScore(html,shop_address):
    
    
    soup = BeautifulSoup(html, 'html.parser')

    infos =soup.find_all('span',attrs = {'class': "section-info-text"} )
#     print(infos)
    
    if len(infos)== 0:
        return 'X','X','X'
        
    else:
        address =infos[0].text
        gu = shop_address[0] in address
        dong = shop_address[1] in address
        bungi = shop_address[2] in address
        if (gu and dong and bungi) or (dong and bungi):
            match = 'match'
        else:
            match = 'not match'

    
        total_score = soup.find_all('div',attrs = {'class': "gm2-display-2"} )

        if len(total_score)!=0:
            total_score = total_score[0].text
        else:
            total_score = 0
    
    return address, total_score, match

In [23]:
def get_info(html, shop_address):
# shop 주소, 총 별점 갖고오기
    address, total_score, match =get_totalScore(html,shop_address)
    
    if address == 'X':
        return 'X','X','X','X','X',
    
    # 리뷰가 없는 경우    
    if  total_score == 0:
        review_list = []
        score_list= []
    # 리뷰가 있는 경우
    else:
        # 리뷰 text(버튼 )클릭
        review_button = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
        review_button.click()

        time.sleep(2)

        # 스크롤 다운
        try:
            element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
            element.click()

        except Exception as err:
            print(err)

        i = 0
        while  True:
            i += 1
            html = driver.find_element_by_tag_name('html')
            time.sleep(SCROLL_PAUSE_TIME)

            text1 = html.text
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.find_element_by_tag_name('html')
            text2 = html.text

            if text1 == text2:
                break
            else:
                continue

        review_html = driver.page_source

        review_list, score_list=get_review(review_html)    

    return address, total_score, review_list, score_list, match

In [24]:
def get_review(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 개별 리뷰
    total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )

    review_list=[]
    for i, review in enumerate(total_review):
        review_list.append(review.text)

    #### 개별 별점
    scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )

    score_list = []
    for score in scores:
        score_list.append(score.attrs['aria-label'])

    return review_list, score_list

In [71]:
def start_crwarling(k, forpet_hash,shop_name, shop_address):
    x_cnt = 0
    match_cnt = 0
    not_match_cnt = 0
    print(k, shop_name, shop_address)

    # shop 이름 입력
    element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
    element.clear()
    
    search_word = shop_name + ' ' +shop_address[0]
    element.send_keys(search_word)
    element.submit()
    
    search_button = driver.find_element_by_xpath('//button[@class="searchbox-searchbutton"]')
    search_button.click()
    
    time.sleep(SCROLL_PAUSE_TIME)

    # 키워드 입력 후, dict로 인덱스와 match 여부 받아오기
    select_shop_html = driver.page_source
    shop_select_dict= find_shop_by_address(select_shop_html, shop_address)
    
    # shoh이 여러개인 경우
    if len(shop_select_dict) != 0:
        print(f'shop이 {len(shop_select_dict)}개입니다')
        for index, match in shop_select_dict.items():
            shop_index =  2 * int(index) -1
#             print('shop_index', shop_index)
            match = match

            time.sleep(5)

            select_shop = driver.find_element_by_xpath(f'//div[@class = "section-layout section-scrollbox scrollable-y scrollable-show section-layout-flex-vertical"]/div/div[{shop_index}]/div[1]/div[1]')
            
            select_shop.click()

            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.page_source

            address, total_score, review_list, score_list, match =get_info(html, shop_address)
            print(address, total_score, len(review_list), len(score_list), match )
    

            if match == 'match':
                match_cnt += 1
                total_table_insert(match_total_table, forpet_hash, shop_name, address, total_score, match)
                review_table_insert(match_review_table, forpet_hash, shop_name, review_list, score_list, match)

            else:
                not_match_cnt +=1
                total_table_insert(not_match_total_table, forpet_hash, shop_name, address, total_score, match)
                review_table_insert(not_match_review_table, forpet_hash, shop_name, review_list, score_list, match)

              
            time.sleep(SCROLL_PAUSE_TIME)
            driver.back()  


    # shop이 1개 인경우
    else:
        print('shop이 1개입니다')
        html = driver.page_source

        address, total_score, review_list, score_list, match =get_info(html, shop_address)

        print(address, total_score, len(review_list), len(score_list), match )
        
        if match == 'X':
            x_cnt += 1
            x_table_insert(x_table, forpet_hash,shop_name)
        elif match == 'match':
            match_cnt += 1
            total_table_insert(match_total_table, forpet_hash, shop_name, address, total_score, match)
            review_table_insert(match_review_table, forpet_hash, shop_name, review_list, score_list, match)
        else:
            not_match_cnt +=1
            total_table_insert(not_match_total_table, forpet_hash, shop_name, address, total_score, match)
            review_table_insert(not_match_review_table, forpet_hash, shop_name, review_list, score_list, match)
    
#         elif match == 'match':
#             match_total_table =total_table_insert(match_total_table, forpet_hash, shop_name, address, total_score, match)
#             print('match_total_table', len(match_total_table))

#             match_review_table =review_table_insert(match_review_table, forpet_hash, shop_name, review_list, score_list, match)
#             print('match_review_table', len(match_review_table))
      
#         else:
#             not_match_total_table=total_table_insert(not_match_total_table, forpet_hash, shop_name, address, total_score, match)
#             print('not_match_total_table', len(not_match_total_table))
      
#             not_match_review_table=review_table_insert(not_match_review_table, forpet_hash, shop_name, review_list, score_list, match)
#             print('not_match_review_table', len(not_match_review_table))


    driver.back()  
     
#     try:
#         back_button = driver.find_element_by_xpath('//button[@class="ozj7Vb3wnYq__action-button-clickable"]')
#         back_button.click()
#     except:
#         pass
    return x_cnt, match_cnt, not_match_cnt    
    

### 2.2. table
#### total table

In [72]:
not_match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'total_score', 'match'])
not_match_total_table

,forpet_hash,shop_name,shop_address,total_score,match


In [73]:
match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'total_score', 'match'])
match_total_table

,forpet_hash,shop_name,shop_address,total_score,match


In [74]:
def total_table_insert(total_table, forpet_hash, shop_name, address, total_score, match):
    if total_score != 0:
        total_table.loc[len(total_table)+1] = [forpet_hash, shop_name, address, total_score, match]
    else:
        total_table.loc[len(total_table)+1] = [forpet_hash, shop_name, address,np.nan, match]
    return total_table

#### review table

In [75]:
not_match_review_table= pd.DataFrame(columns = ['forpet_hash', 'num', 'review', 'score', 'match'])
not_match_review_table

,forpet_hash,num,review,score,match


In [76]:
match_review_table= pd.DataFrame(columns = ['forpet_hash', 'num', 'review', 'score', 'match'])
match_review_table

,forpet_hash,num,review,score,match


In [77]:
def review_table_insert(review_table, forpet_hash, shop_name, review_list, score_list, match):
    if len(review_list) != 0:
        for i, (review, score, match) in enumerate(zip(review_list, score_list, match)):
            review_table.loc[i+1+len(review_table)] = [forpet_hash, i, review, score, match]
       
    else:
        review_table.loc[1+len(review_table)] = [forpet_hash, 1, np.nan, np.nan , match]    
    
    return review_table

In [78]:
x_table = pd.DataFrame(columns = ['forpet_hash', 'shop_name'])
x_table

,forpet_hash,shop_name


In [79]:
def x_table_insert(x_table, forpet_hash, shop_name):
    x_table.loc[1+len(x_table)] = [forpet_hash, shop_name]
    return x_table

### 2.3. 일부로 연습

In [80]:
ordinary_table = pd.read_csv('mysql_forpet_shop.csv', index_col = 0)

In [45]:
ordinary_table.head()

,forpet_hash,address_name,place_name
0,4b793eb834462b029e435eaeb49433efc61755c9e15a24...,서울 종로구 평창동 530-8,쿠나이앤티
1,f5169c15a126e4059f0a89bb80fccd236693d5056d24ae...,서울 종로구 창신동 468-1,애견용품할인매장
2,aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb...,서울 종로구 창신동 436-64,펫클럽 청계점
3,2806f173e3a783ed14d74c9a83da2f62c34f45d470e893...,서울 종로구 홍지동 103-24,나니스펫푸드
4,aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c...,서울 종로구 필운동 137-5,서촌양품


In [46]:
first_shop =ordinary_table.head(20)

In [47]:
first_shop

,forpet_hash,address_name,place_name
0,4b793eb834462b029e435eaeb49433efc61755c9e15a24...,서울 종로구 평창동 530-8,쿠나이앤티
1,f5169c15a126e4059f0a89bb80fccd236693d5056d24ae...,서울 종로구 창신동 468-1,애견용품할인매장
2,aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb...,서울 종로구 창신동 436-64,펫클럽 청계점
3,2806f173e3a783ed14d74c9a83da2f62c34f45d470e893...,서울 종로구 홍지동 103-24,나니스펫푸드
4,aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c...,서울 종로구 필운동 137-5,서촌양품
5,8aa63cf25e3842284a45ab5d4f603e816dba32c9406b88...,서울 종로구 이화동 191-2,푸드마스터그룹
6,c923e036035e04fc590dfe181702bdff963f7fe6e0d442...,서울 종로구 창신동 142-35,스프링펫 창신점
7,275c9b2f484f215bf7da6e27c0f2ea8696469033657186...,서울 종로구 창신동 327-2,다이렉트 펫
8,52f7350baf9645ba80f749b3d86f5e746cf46aaf1265ba...,서울 종로구 인사동 15,스투키펫
9,8e35f7a0a73e558271ae3dbec0ad40322ed25b73045eb6...,서울 서대문구 홍제동 279-27,반려동물용품사료 할인마트


In [69]:
len(ordinary_table)

8582

In [81]:
SCROLL_PAUSE_TIME = 2
all_cnt = 0
for k in range(len(ordinary_table)):
    forpet_hash, address_name, place_name= ordinary_table.loc[k] 
  
    shop_address =address_name.split(' ')[1:]
    x_cnt, match_cnt, not_match_cnt =  start_crwarling(k, forpet_hash, place_name, shop_address)
    cur_cnt = x_cnt + match_cnt + not_match_cnt
    print('cur_cnt',cur_cnt)
    all_cnt += cur_cnt
    print('k+1:', k+1, 'all_cnt:',all_cnt)
    print('match_total_table:',len(match_total_table))
    print('not_match_total_table:',len(not_match_total_table))
    print('x_table:', len(x_table))
    print('match_review_table:',len(match_review_table))
    print('not_match_review_table:',len(not_match_review_table))
    print('\n')
    
    try:
        back_button = driver.find_element_by_xpath('//button[@class="ozj7Vb3wnYq__action-button-clickable"]')
        back_button.click()
    except:
        pass

0 쿠나이앤티 ['종로구', '평창동', '530-8']
shop이 1개입니다
서울특별시 종로구 평창동 0 0 0 not match
cur_cnt 1
k+1: 1 all_cnt: 1
match_total_table: 0
not_match_total_table: 1
x_table: 0
match_review_table: 0
not_match_review_table: 1


1 애견용품할인매장 ['종로구', '창신동', '468-1']
address list 0 창신동 315-1번지 1층 종로구 서울특별시 KR
address list 1 창신동 436-64번지
address list 2 금호동1가 513번지 성동구 서울특별시 KR
shop이 2개입니다
창신동 315-1번지 1층 종로구 서울특별시 KR 0 0 0 not match
서울특별시 종로구 창신동 436-64번지 0 0 0 not match
cur_cnt 2
k+1: 2 all_cnt: 3
match_total_table: 0
not_match_total_table: 3
x_table: 0
match_review_table: 0
not_match_review_table: 3


2 펫클럽 청계점 ['종로구', '창신동', '436-64']
shop이 1개입니다
서울특별시 종로구 창신동 436-64번지 일신파워빌딩 0 0 0 match
cur_cnt 1
k+1: 3 all_cnt: 4
match_total_table: 1
not_match_total_table: 3
x_table: 0
match_review_table: 1
not_match_review_table: 3


3 나니스펫푸드 ['종로구', '홍지동', '103-24']
shop이 1개입니다
홍지동 103-24번지 1층 종로구 서울특별시 KR 0 0 0 match
cur_cnt 1
k+1: 4 all_cnt: 5
match_total_table: 2
not_match_total_table: 3
x_table: 0
match_review_table:

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class = "section-layout section-scrollbox scrollable-y scrollable-show section-layout-flex-vertical"]/div/div[5]/div[1]/div[1]"}
  (Session info: chrome=80.0.3987.132)


In [ ]:

print('match_total_table\n',match_total_table)
print('not_match_total_table\n',not_match_total_table)
print('match_review_table\n',match_review_table)
print('not_match_review_table\n',not_match_review_table)
print('x_table\n', x_table)